# Behind the scenes: Accessing webpages with user log-in

In the first instance, the aim of this project is to learn how to retrieve an html page that is protected by a user login portal. UPDATE: Got that sorted

Eventually, the intention is to access a government database of court cases and to automatically retrieve information on 10k+ court cases based on a list of case numbers.

In [22]:
import mechanicalsoup
import requests
import pandas as pd
import json

In [23]:
# import list of case numbers from csv or wherever
queries = ['CL-2014-000636',
'CL-2015-000128',
'BL-2018-002514',
'CR-2019-003497',
'LM-2019-000044',
'BL-2018-002522',
'HC-2017-002130',
'HC-2015-004346',
'CL-2015-000634',
'CL-2018-000709']

In [26]:
cases_out = []
username = "markw@decoded.com"
password = "ST$RD£Wva11ey" 
browser = mechanicalsoup.StatefulBrowser(raise_on_404=True)
    
for case in queries:
    print("running case: ", case)
    try:
        url = "https://efile.cefile-app.com/officecopies/filing/search?caseNumber="+str(case)+"&formToken=1582146479269"
        browser.open(url)
    except:
        print("Could not open case: ", case)
        continue;

    # Only seem to need this on the first query, move out of loop
    try:
        form = browser.select_form('form[id=loginForm]')
        form['username'] = username
        form['password'] = password
        response = browser.submit_selected()
        print("Submitted username and password")
    except:
        #print("No login needed")
        response = browser.open(url)
    # Submit the form and print the resulting page text
    
    
    # Set up a dataframe to feed case event into
    #case_log = pd.DataFrame(columns = ["Event", "Submitted Date", "Filed Date", "Type", "Description"])
    
    try:
        response.text.split('case number entered')[1]
        print('case does not exist')
        cases_out.append({"case_nr" : case, "error" : "case does not exist or is not available"})
    except:
    
        try:
            response.text.split('<tbody></tbody>')[1]
            print('No records found')
            #case_log = case_log.append(pd.DataFrame.from_dict({'Event' : ['N/A']}), sort = False)
            cases_out.append({"case_nr" : case, "events" : "No records found"})

        except:
            case_events = []
            for table_row in response.text.split("Case Event Log")[1].split('tbody>')[1].split('<tr>')[1:]:
                eventNr = table_row.split('<td>')[1][:-5]
                subDate = table_row.split('<td>')[2].split('>')[1].split('<')[0]
                fileDate = table_row.split('<td>')[3].split('>')[1].split('<')[0]
                entryType = table_row.split('<td>')[4][:-5]
                description = table_row.split('<td>')[5][:-5]

                current_event = {'EventNr': [eventNr], 
                            'Submitted Date': [subDate], 
                            'Filed Date': [fileDate], 
                            'Type' : [entryType],
                            'Description' : [description] }

                case_events.append(current_event)
            print("Case retrieved")

            cases_out.append({"case_nr" : case, "events" : case_events})


running case:  CL-2014-000636
Submitted username and password
Case retrieved
running case:  CL-2015-000128
No login needed
Case retrieved
running case:  BL-2018-002514
No login needed
Case retrieved
running case:  CR-2019-003497
No login needed
No records found
running case:  LM-2019-000044
No login needed
case does not exist
running case:  BL-2018-002522
No login needed
No records found
running case:  HC-2017-002130
No login needed
Case retrieved
running case:  HC-2015-004346
No login needed
Case retrieved
running case:  CL-2015-000634
No login needed
Case retrieved
running case:  CL-2018-000709
No login needed
Case retrieved


In [27]:
with open('case_event_logs.json', 'w') as fout:
    json.dump(cases_out , fout)

# Explore how storing in json would work

In [20]:
try:
    url = "https://efile.cefile-app.com/officecopies/filing/search?caseNumber="+"CR-2019-003497"+"&formToken=1582146479269"
    browser.open(url)
except:
    print("Could not open case: ", case)

# Only seem to need this on the first query, move out of loop
try:
    form = browser.select_form('form[id=loginForm]')
    form['username'] = username
    form['password'] = password
    response = browser.submit_selected()
    print("Submitted username and password")
except:
    print("No login needed")
    response = browser.open(url)
# Submit the form and print the resulting page text


# Set up a dataframe to feed case event into
#case_log = pd.DataFrame(columns = ["Event", "Submitted Date", "Filed Date", "Type", "Description"])




No login needed


In [21]:
response.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">\r\n<html lang="" xml:lang="">\r\n<head>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta http-equiv="content-type" content="text/html; charset=utf-8">\r\n\r\n<link rel="shortcut icon" href="/include/images/favicon.ico" type="image/x-icon"/>\r\n<link rel="icon" href="/include/images/favicon.ico" type="image/x-icon"/>\r\n\r\n<link rel="apple-touch-icon" sizes="57x57" href="/include/images/iOS/57.png"/>\r\n<link rel="apple-touch-icon" sizes="72x72" href="/include/images/iOS/72.png"/>\r\n<link rel="apple-touch-icon" sizes="114x114" href="/include/images/iOS/114.png"/>\r\n<link rel="apple-touch-icon" sizes="144x144" href="/include/images/iOS/144.png"/>\r\n\r\n<link rel="stylesheet" type="text/css" href="/include/css/normalize.css" media="all">\r\n<link rel="stylesheet" type="text/css" href="/include/css/superfish-1.4.8.css" media="all">\r\n<link rel="stylesheet" type="text/css" hre